In [14]:
import datetime
import numpy as np
import pandas as pd

from datacompy import Compare, render

from typing import Any, Dict, List, Optional, Union, cast


file1 = "listings.csv"
file2 = "listings test.csv"

def read_data(file):
    """Reads data from an uploaded file based on its extension."""
    if file.endswith('.xlsx'):
        return pd.read_excel(file)
    elif file.endswith('.csv'):
        return pd.read_csv(file)
    else:
        raise ValueError("Unsupported file format. Please upload a CSV or Excel file.")

class DatalyCompare(Compare):
    def Regression_report(
        self,
        sample_count: int = 10,
        html_file: Optional[str] = None,
    ) -> str:
        if self.df1.empty or self.df2.empty:
            raise ValueError("One or both dataframes are empty. Comparison requires non-empty dataframes.")

        def df_to_str(pdf: pd.DataFrame) -> str:
            if not self.on_index:
                pdf = pdf.reset_index(drop=True)
            return pdf.to_string()
        
        current_time = datetime.datetime.now()
        formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

        # Header
        report = ("Dataly Regression Test Output")
        report += "\n"
        report += "--------------------"
        report += "\n"
        report += "The report was generated on " + formatted_time
        report += "\n\n"
        report += "DataFrame Summary"
        report += "\n"
        report += "-----------------"
        report += "\n\n"
        df_header = pd.DataFrame(
            {
                "DataFrame": [self.df1_name, self.df2_name],
                "Columns": [self.df1.shape[1], self.df2.shape[1]],
                "Rows": [self.df1.shape[0], self.df2.shape[0]],
            }
        )
        report += df_header[["DataFrame", "Columns", "Rows"]].to_string()
        report += "\n\n"

        # Column Summary
        report += render(
            "column_summary.txt",
            len(self.intersect_columns()),
            len(self.df1_unq_columns()),
            len(self.df2_unq_columns()),
            self.df1_name,
            self.df2_name,
        )
        
        report += "Columns with all Null values: \n" 
        df1_null_columns = self.df1.columns[self.df1.isna().all()].tolist()
        df2_null_columns = self.df2.columns[self.df2.isna().all()].tolist()
        if df1_null_columns :
            null_columns_df1 = pd.DataFrame(df1_null_columns, columns=[f"Columns with all Null values in {self.df1_name}"])
            report += null_columns_df1.to_string(index=False) + "\n\n"
        else:
            report += f"No columns with all null values in {self.df1_name}.\n"

        if df2_null_columns:
            null_columns_df2 = pd.DataFrame(df2_null_columns, columns=[f"Columns with all Null values in {self.df2_name}"])
            report += null_columns_df2.to_string(index=False) + "\n\n"
        else:
            report += f"No columns with all null values in {self.df2_name}.\n\n"

        # Row Summary
        if self.on_index:
            match_on = "index"
        else:
            match_on = ", ".join(self.join_columns)
        report += render(
            "row_summary.txt",
            match_on,
            self.abs_tol,
            self.rel_tol,
            self.intersect_rows.shape[0],
            self.df1_unq_rows.shape[0],
            self.df2_unq_rows.shape[0],
            self.intersect_rows.shape[0] - self.count_matching_rows(),
            self.count_matching_rows(),
            self.df1_name,
            self.df2_name,
            "Yes" if self._any_dupes else "No",
        )

        match_stats = []
        match_sample = []
        match_full_list = []
        any_mismatch = False
        for column in self.column_stats:
            if not column["all_match"]:
                any_mismatch = True
                match_stats.append(
                    {
                        "Column": column["column"],
                        f"{self.df1_name} dtype": column["dtype1"],
                        f"{self.df2_name} dtype": column["dtype2"],
                        "# Unequal": column["unequal_cnt"],
                        "Max Diff": column["max_diff"],
                        "# Null Diff": column["null_diff"],
                    }
                )
                if column["unequal_cnt"] > 0:
                    match_sample.append(
                        self.sample_mismatch(
                            column["column"], sample_count, for_display=True
                        )
                    )
                    match_full_list.append(
                        self.ful_list_single_mismatch(
                            column["column"]
                        )
                    )
                  

        if any_mismatch:
            report += "Columns with Unequal Values or Types\n"
            report += "------------------------------------\n"
            report += "\n"
            df_match_stats = pd.DataFrame(match_stats)
            df_match_stats.sort_values("Column", inplace=True)
            # Have to specify again for sorting
            report += df_match_stats[
                [
                    "Column",
                    f"{self.df1_name} dtype",
                    f"{self.df2_name} dtype",
                    "# Unequal",
                    "Max Diff",
                    "# Null Diff",
                ]
            ].to_string()
            report += "\n\n"
            
            report += "Key Summary of Unequal Values\n"
            report += "-------------------------------\n"
            report += "\n"
            if match_full_list:
              
                report += df_to_str(pd.concat(match_full_list, axis=0))
                report += "\n\n"

            if sample_count > 0:
                report += "Sample Rows with Unequal Values\n"
                report += "-------------------------------\n"
                report += "\n"
                for sample in match_sample:
                    report += df_to_str(sample)
                    report += "\n\n"

        return report

    def ful_list_single_mismatch(
        self, column: str
    ) -> pd.DataFrame:
        """Returns a all sub-dataframe which contains the identifying
        columns, and df1 and df2 versions of the column.
        """
        col_match = self.intersect_rows[column + "_match"]
        full_list = self.intersect_rows[~col_match]
        return_cols = self.join_columns + [
            column + "_" + self.df1_name,
            column + "_" + self.df2_name,
        ]
        return_df = full_list[return_cols]
        return_df.columns = pd.Index(
            self.join_columns
            + [
                column + " (" + self.df1_name + ")",
                column + " (" + self.df2_name + ")",
            ]
        )
        group_sizes  = return_df.groupby([column + " (" + self.df1_name + ")", column + " (" + self.df2_name + ")" ]).size().reset_index()
        group_sizes.rename(
        columns={
            f"{column} ({self.df1_name})": self.df1_name,
            f"{column} ({self.df2_name})": self.df2_name,
            0: 'Count'
            }, inplace=True
        )
        
        # Add a column for the mismatched column name
        group_sizes['Column'] = column
        
        # Order columns
        order = ['Column', self.df1_name, self.df2_name, 'Count']
        return group_sizes[order]

df1 = read_data(file1)
df2 = read_data(file2)

In [32]:
join_columns = ["id"]

compare = DatalyCompare(
df1,
df2,
join_columns=join_columns, #You can also specify a list of columns
abs_tol=0.0001,
rel_tol=0,
df1_name="original",
df2_name="new")

print(compare.Regression_report())



Dataly Regression Test Output
--------------------
The report was generated on 2024-06-24 15:47:36

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0  original       92  3818
1       new       92  3818

Column Summary
--------------

Number of columns in common: 92
Number of columns in original but not in new: 0
Number of columns in new but not in original: 0

  Columns with all Null values in original
0  license                                

  Columns with all Null values in new
0  license                           

Row Summary
-----------

Matched on: id
Any duplicates on match values: No
Absolute Tolerance: 0.0001
Relative Tolerance: 0
Number of rows in common: 3,818
Number of rows in original but not in new: 0
Number of rows in new but not in original: 0

Number of rows with some compared columns unequal: 46
Number of rows with all compared columns equal: 3,772

Columns with Unequal Values or Types
------------------------------------

        Column original dt

In [11]:
import pandas as pd

# Sample DataFrame
data = {
    'Column1': ['A', 'B', 'A', 'A', 'B', 'C', 'C', 'C', 'B', 'A'],
    'Column2': ['X', 'X', 'Y', 'Y', 'Z', 'Z', 'X', 'Y', 'Y', 'Z'],
    'Column3': [1, 1, 1, 2, 2, 2, 3, 3, 3, 1]
}

df = pd.DataFrame(data)
print(df)

# Group by multiple columns and count
grouped = df.groupby(['Column1', 'Column2', 'Column3']).size()
print(grouped)

  Column1 Column2  Column3
0       A       X        1
1       B       X        1
2       A       Y        1
3       A       Y        2
4       B       Z        2
5       C       Z        2
6       C       X        3
7       C       Y        3
8       B       Y        3
9       A       Z        1
Column1  Column2  Column3
A        X        1          1
         Y        1          1
                  2          1
         Z        1          1
B        X        1          1
         Y        3          1
         Z        2          1
C        X        3          1
         Y        3          1
         Z        2          1
dtype: int64


In [30]:
# Convert to DataFrame and rename the count column
grouped_df = grouped.reset_index()
print(grouped_df)


  Column1 Column2  Column3  0
0       A       X        1  1
1       A       Y        1  1
2       A       Y        2  1
3       A       Z        1  1
4       B       X        1  1
5       B       Y        3  1
6       B       Z        2  1
7       C       X        3  1
8       C       Y        3  1
9       C       Z        2  1


In [15]:
data1 = {'A': [1, 2, 3], 'B': [4, 5, 6]}
data2 = {'A': [1, 2, 3], 'B': [4, 5, 7]}
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
comparison = DatalyCompare(df1, df2, join_columns=['A'])
report = comparison.Regression_report()
print(report)

Dataly Regression Test Output
--------------------
The report was generated on 2024-06-24 17:25:40

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1        2     3
1       df2        2     3

Column Summary
--------------

Number of columns in common: 2
Number of columns in df1 but not in df2: 0
Number of columns in df2 but not in df1: 0

Columns with all Null values: 
No columns with all null values in df1.
No columns with all null values in df2.

Row Summary
-----------

Matched on: a
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 3
Number of rows in df1 but not in df2: 0
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 1
Number of rows with all compared columns equal: 2

Columns with Unequal Values or Types
------------------------------------

  Column df1 dtype df2 dtype  # Unequal  Max Diff  # Null Diff
0      b     int64     int64          1       1.0  